In [2]:
# By convention, import all modules at the top of the notebook so they don't get in the way later



<!--this prints a function in-line with the surrounding text-->
<!--to make it appear, the html comment marks must be removed from the following two lines-->
<!--
This is what $f(x)$ looks like for this example:
$$f(x) = \int_x^\infty y^2\, dy$$
-->

<!--this is a markdown comment-->
<!--this is heading one text-->
# CW04 Prime Number Generator
<!--this is using the Latex equation syntax to print our names in italics-->
$Nengyin\,(Helen)\,Zhu,\, Lance\, Clifner,\, Alexis\, Ford$ 

<!--this is normal body text-->
September 20, 2016

This is Classwork Assignment number 04.  In this project, we are officially switching from python 2 to python 3, and all further classwork will be completed in python 3.

For this project, we are creating a python script that computes a list of all the prime numbers between 1 and *N*, where *N* is a number passed on the command line to the prime number python script.

The algorithm we have implemented to calculate the prime number is the <a href="https://en.wikipedia.org/wiki/Sieve_of_Eratosthenes">Sieve of Eratosthenes</a>.  You can follow this link to see a complete description of the algorithm, along with an animation that visually demonstrates the algorithm's steps.  

In brief, the algorithm creates a sequence of all integers between 2 and *N*.  The algorithm then starts with the number 2 (the smallest prime number), and eliminates all multiples of 2.  The algorithm then finds the next number in the sequence that has not yet been eliminated (this is the next prime number in the sequence), and proceeds to remove all multiples of that new number from the sequence.  Lather, rinse, and repeat until *N* is reached.

Because we have a mathmetician in our group, we know that you can stop your search for primes once you have reached $\sqrt{N}$.

Our hero, Eratosthenes of Cyrene: 
![Alt text](http://www.conservapedia.com/images/b/bc/Eratosthenes.jpg "Eratosthenes of Cyrene")

Next is our cut at a python implementation of Eratosthenes Sieve.

In [ ]:
# insert the prime number function here

# Design Considerations


## Conclusion

Happy notebooking.